In [0]:
# a. Create a Storage Account in the azure portal
# I have created new resource group for week 15 assignment and inside this I have created one
# Storage Account week15assignstorage with Hierarchical namespace Enabled.

In [0]:
# b. Create and launch a Databricks Workspace in the azure portal
# Created Databricks Workspace week15-tt-databricks-ws

In [0]:
# I have uploaded 2 datasets a. bike_trip.csv and b. donation.csv both are more than 1 GB in size

In [0]:
# Create a mount point /mnt/week15assignmentdb to access the files in
# the container - week15inputdatasets – To create mount point I have first
# created one secret scope to keep azure storage account key and reading
# the same while creating mount point .

In [0]:
# Create Data frames by reading the data present in the storage account through the mount point created in the previous step

In [0]:
dbutils.fs.mount(source='wasbs://week15inputdatasets@week15assignstorage.blob.core.windows.net', 
mount_point = '/mnt/week15assignmentdb',
extra_configs={'fs.azure.account.key.week15assignstorage.blob.core.windows.net' : dbutils.secrets.get("Week15-tt-secret-scope","week15assignstoragekey")})

In [0]:
bike_schema ='id long, rideable_type string, started_at string ,ended_at string ,start_station_name string ,start_station_id string ,end_station_name string ,end_station_id string ,start_lat string ,start_lng string ,end_lat string ,end_lng string ,member_casual string ,ride_length string ,day_of_week_start string ,day_of_week_end string'

bike_df = spark.read.format("csv").option("header","true").schema(bike_schema).load("dbfs:/mnt/week15assignmentdb/bike_trip")

In [0]:
donation_schema="project_id string, donation_id string, donar_id string, donation_included_optional_donation string, donation_amount string, donor_cart_sequence string, donation_received_date string"

donation_df = spark.read.format("csv").schema(donation_schema).option("header","false").load("dbfs:/mnt/week15assignmentdb/donation")

In [0]:
# create a Database and create delta tables on the data stored in the Storage account

In [0]:
%sql
Create database if not exists week15assigndb

In [0]:
# Create Spark tables in Parquet format

In [0]:
bike_df.write.mode("overwrite").partitionBy("day_of_week_start").format("parquet").save("/mnt/week15assignmentdb/parquet/bikes.parquet"
# Due to partition By column, we have 7 folders generated in a storage account.

In [0]:
# Creating a parquet table for bike_trip.csv

In [0]:
%sql
create table week15assigndb.bikesparquet using parquet location "/mnt/week15assignmentdb/parquet/bikes.parquet/*"

In [0]:
%sql
select * from week15assigndb.bikesparquet

In [0]:
donation_df.write.mode("overwrite").format("parquet").save("/mnt/week15assignmentdb/parquet/donations.parquet")

In [0]:
# Creating Parquet Table for donations.csv

In [0]:
%sql
create table week15assigndb.donationsparquet using parquet location "/mnt/week15assignmentdb/parquet/donations.parquet/*"

In [0]:
%sql
select * from week15assigndb.donationsparquet

In [0]:
# b. Create Spark tables in Delta format
# Creating Delta table for bike_trip.csv

In [0]:
bike_df.write.mode("overwrite").partitionBy("day_of_week_start").format("delta").save("/mnt/week15assignmentdb/delta/bikes.delta")

In [0]:
%sql
create table week15assigndb.bikesdelta using delta location "/mnt/week15assignmentdb/delta/bikes.delta/*"

In [0]:
%sql
select * from week15assigndb.bikesdelta

In [0]:
# Creating Delta table for donations.csv

In [0]:
donation_df.write.mode("overwrite").format("delta").save("/mnt/week15assignmentdb/delta/donations.delta")

In [0]:
%sql
create table week15assigndb.donationsdelta using delta location "/mnt/week15assignmentdb/delta/donations.delta/*"

In [0]:
%sql
select * from week15assigndb.donationsdelta

In [0]:
# How is the Parquet format structuring of files different from that of Delta
# format? Give a detailed explanation with appropriate examples and
# diagrams.
# There is a difference of _delta_log folder between parquet and delta format so
# Delta = parquet + _delta_log
# DeltaLog contains information about every transaction that modifies our data. It guarantees to Spark
# a single source of truth, which means that every time a query is executed against a Delta table, the
# engine checks the log to see which data files should be read.

In [0]:
%sql
describe history week15assigndb.bikesdelta
-- DESCRIBE HISTORY is only supported for Delta tables – on parquet table we cannot run the describe history command because of the delta_log file

In [0]:
# 5. Create a delta table in a single step while writing the data to the dataframe using saveAsTable option

In [0]:
bike_df.write.mode("overwrite").partitionBy("day_of_week_start").format("delta").option("path","/mnt/week15assigndb/deltalatest/bikes.delta").saveAsTable("week15assigndb.bikesdeltatable")

In [0]:
donation_df.write.mode("overwrite").format("delta").option("path","/mnt/week15assigndb/deltalatest/donations.delta").saveAsTable("week15assigndb.donationsdeltatable")

In [0]:
%sql
select * from week15assigndb.donationsdeltatable

In [0]:
# Insert the data into the Delta tables using the following 3 approaches

In [0]:
# Insert

In [0]:
%sql
insert into week15assigndb.bikesdeltatable values
('111111111','electric_bike','2021-10-01 15:48:22','2021-10-01 15:48:22','Clark St & Leland Ave','13136','Wilton Ave & Diversey Pkwy','TA1306000014','41.9541235','-87.65478743','41.90096','-87.644336','casual','28.716666666666665','Fri','Fri')

In [0]:
#  Append

In [0]:
bike_schema_new ='id long, rideable_type string, started_at string,ended_at string ,start_station_name string ,start_station_id string,end_station_name string ,end_station_id string ,start_lat string,start_lng string ,end_lat string ,end_lng string ,member_casual string,ride_length string ,day_of_week_start string ,day_of_week_end string'

bikenew_df = spark.read.format("csv").option("header","true").schema(bike_schema_new).load("dbfs:/mnt/week15assignmentdb/append_bikes.csv")

In [0]:
bikenew_df.write.mode("append").partitionBy("day_of_week_start").format("delta").option("path","/mnt/week15assigndb/deltalatest/bikes.delta").save()

In [0]:
# Copy

In [0]:
%sql
copy into week15assigndb.bikesdeltatable from
'dbfs:/mnt/week15assignmentdb/copy_bikes.csv' fileformat=CSV format_options('header'='true')
-- If both the schemas are the same, the above command should work

In [0]:
# Schema enforcement, also known as schema validation, is a safeguard in Delta Lake that
# ensures data quality by rejecting writes to a table that do not match the table's schema. Like
# the front desk manager at a busy restaurant that only accepts reservations, it checks to see
# whether each column in data inserted into the table is on its list of expected columns (in other
# words, whether each one has a "reservation"), and rejects any writes with columns that aren't
# on the list.
# If we add option("mergeSchema","true") then only the delta table will allow for append
# operation in case of the extra column.

In [0]:
bike_schema_mismatched ='id long, rideable_type string, started_at string,ended_at string ,start_station_name string ,start_station_id string,end_station_name string ,end_station_id string ,start_lat string,start_lng string ,end_lat string ,end_lng string ,member_casual string,ride_length string ,day_of_week_start string ,day_of_week_end string,test string'

bikemismatched_df = spark.read.format("csv").option("header","true").schema(bike_schema_mismatched).load("dbfs:/mnt/week15assignmentdb/schema_mismatch_bikes.csv")

In [0]:
bikemismatched_df.write.mode("append").partitionBy("day_of_week_start").format("delta").option("path","/mnt/week15assigndb/deltalatest/bikes.delta").option("mergeSchema","true").save()

In [0]:
# Depict the internal working of update and delete
# operation by updating and deleting records of the data
# # present in the storage account.

In [0]:
# Once Delta Lake has executed the UPDATE successfully, it adds a commit in the transaction
# log indicating that the new data file will be used in place of the old one from now on. The old
# data file is not deleted, though. Instead, it's simply "tombstoned" — recorded as a data file
# that applied to an older version of the table, but not the current version. Delta Lake is able to
# # use it to provide data versioning and time travel.

In [0]:
%sql
update week15assigndb.bikesdeltatable set day_of_week_start='Fri' where id = '9999999999' and test='test'

In [0]:
# DELETE works just like UPDATE under the hood. Delta Lake makes two scans of the data:
# the first scan is to identify any data files that contain rows matching the predicate condition.
# The second scan reads the matching data files into memory, at which point Delta Lake deletes
# the rows in question before writing out the newly clean data to disk.
# After Delta Lake completes a DELETE operation successfully, the old data files are not
# deleted — they're still retained on disk, but recorded as "tombstoned" (no longer part of the
# active table) in the Delta Lake transaction log. Remember, those old files aren't deleted
# immediately because you might still need them to time-travel back to an earlier version of the
# table. If you want to delete files older than a certain time period, you can use
# the VACUUM command.

In [0]:
# Apply NOT NULL and CHECK constraints on the data
# and demonstrate the behaviour when data violating the
# constraints are inserted into the delta table.

In [0]:
%sql
alter table week15assigndb.bikesdeltatable alter column rideable_type SET
not null

In [0]:
%sql
alter table week15assigndb.bikesdeltatable add constraint status_ck
check(rideable_type in ("electric_bike","docked_bike","classic_bike"))

In [0]:
# here have been several changes being made to the
# table. Say you are required to present the original data
# without any changes, restore the table to its first
# version.

In [0]:
%sql
restore table week15assigndb.bikesdeltatable to version as of 0